In [9]:

import pandas as pd
import numpy as np

df_all = pd.read_csv("cortical_metrics_1.4.csv")

# Exc and PV have sufficient number of cells, so we'll filter out non-V1 Exc and PV.
exclude = (
    df_all["cell_type"].isnull()
    | df_all["cell_type"].str.contains("EXC")
    | df_all["cell_type"].str.contains("PV")
) & (df_all["ecephys_structure_acronym"] != "VISp")

df = df_all[~exclude]
print(f"Original: {df_all.shape[0]} cells,   filtered: {df.shape[0]} cells")

# export V1 only dataframe
df.to_csv("cortical_metrics_1.4_V1.csv")


/local1/local/miniconda/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3229: DtypeWarning: Columns (60,62,65,67,69,72,119,123) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Original: 40294 cells,   filtered: 8799 cells


In [3]:

bo_counts = df[df["session_type"] == "brain_observatory_1.1"][
    "cell_type"
].value_counts()
fc_counts = df[df["session_type"] == "functional_connectivity"][
    "cell_type"
].value_counts()

# Some cells have very large values of RF. They are likely not-good fits, so ignore.
df.loc[(df["width_rf"] > 100), "width_rf"] = np.nan
df.loc[(df["height_rf"] > 100), "height_rf"] = np.nan

# define contrast reponse types
types = ["high", "band", "low"]
type_fracs = {}
type_fracs_err = {}
contrast_responsive = df["sig0.01"] == True
contrast_responsive_counts = df[contrast_responsive]["cell_type"].value_counts()
for t in types:
    type_counts = df[(df["best_model"] == t) & contrast_responsive][
        "cell_type"
    ].value_counts()
    n = contrast_responsive_counts
    p = type_counts / n
    type_fracs[t] = p
    type_fracs_err[t] = np.sqrt(p * (1 - p) / n)


# set group statistics, and insert the cell counts for convenience
resource_tables = {}
resource_tables["median"] = df.groupby("cell_type").median()
resource_tables["mean"] = df.groupby("cell_type").mean()
resource_tables["sem"] = df.groupby("cell_type").sem()

for t in resource_tables.values():
    # insert counts and fractional elements that is common for all 3 tables
    t.insert(0, "n_cells_BO", bo_counts)
    t.insert(1, "n_cells_FC", fc_counts)
    # t.insert(-1, "frac_highpass", type_fracs["high"])
    t["frac_highpass"] = type_fracs["high"]
    t["frac_bandpass"] = type_fracs["band"]
    t["frac_lowpass"] = type_fracs["low"]

# sem is a special case. re-insert the estimated error of the fraction
resource_tables["sem"]["frac_highpass"] = type_fracs_err["high"]
resource_tables["sem"]["frac_bandpass"] = type_fracs_err["band"]
resource_tables["sem"]["frac_lowpass"] = type_fracs_err["low"]


/local1/local/miniconda/local/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/local1/local/miniconda/local/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [5]:
selected_metrics = [
    "n_cells_BO",
    "n_cells_FC",
    "g_osi_dg",
    "g_dsi_dg",
    "pref_sf_sg",
    "pref_tf_dg",
    "firing_rate_rf",
    "firing_rate_dg",
    "firing_rate_sp",
    "width_rf",
    "height_rf",
    "fano_dg",
    "frac_highpass",
    "frac_bandpass",
    "frac_lowpass",
]

# For the full list of metrics, refer to list(df.columns)
#
# See https://allensdk.readthedocs.io/en/latest/visual_coding_neuropixels.html
# for detailed descriptions of the metrics.


In [6]:
print('Median functional responses')
resource_tables["median"][selected_metrics]


Median functional responses


,n_cells_BO,n_cells_FC,g_osi_dg,g_dsi_dg,pref_sf_sg,pref_tf_dg,firing_rate_rf,firing_rate_dg,firing_rate_sp,width_rf,height_rf,fano_dg,frac_highpass,frac_bandpass,frac_lowpass
cell_type,,,,,,,,,,,,,,,
ALL_L1,50,91.0,0.176471,0.108439,0.04,2.0,0.002193,0.402949,2.206366,25.904446,28.653408,3.371775,0.833333,0.166667,NaN
EXC_L23,203,300.0,0.297405,0.189797,0.04,2.0,0.286186,0.657637,0.633934,13.049746,14.679953,2.732950,0.538961,0.285714,0.175325
EXC_L4,301,328.0,0.200445,0.118240,0.04,2.0,1.009873,2.047231,2.121063,25.278849,28.292258,3.305578,0.614973,0.299465,0.085561
EXC_L5,482,493.0,0.189457,0.077074,0.04,2.0,2.411210,4.467382,4.046847,32.846834,37.048498,3.135327,0.709877,0.231481,0.058642
EXC_L6,191,153.0,0.422593,0.084992,0.04,2.0,1.307024,2.707690,1.863628,31.955986,34.750666,3.342077,0.759494,0.177215,0.063291
PV_L23,75,58.0,0.110700,0.091677,0.08,2.0,3.383541,5.196432,7.350043,17.976579,21.951647,6.201904,0.666667,0.166667,0.166667
PV_L4,98,81.0,0.105398,0.073914,0.04,2.0,4.337741,7.718506,8.861297,28.140296,28.833984,6.558140,0.622222,0.222222,0.155556
PV_L5,99,101.0,0.121864,0.062068,0.08,2.0,4.783467,10.248237,8.945708,31.783177,34.507976,6.590634,0.790123,0.148148,0.061728
PV_L6,56,34.0,0.159636,0.057974,0.06,2.0,5.272505,11.862690,9.762731,46.371927,48.528596,5.587174,0.807692,0.153846,0.038462


In [7]:
print('Mean functional responses')
resource_tables["mean"][selected_metrics]


Mean functional responses


,n_cells_BO,n_cells_FC,g_osi_dg,g_dsi_dg,pref_sf_sg,pref_tf_dg,firing_rate_rf,firing_rate_dg,firing_rate_sp,width_rf,height_rf,fano_dg,frac_highpass,frac_bandpass,frac_lowpass
cell_type,,,,,,,,,,,,,,,
ALL_L1,50,91.0,0.328164,0.258430,0.064400,2.340426,0.772524,1.338205,4.605104,2.349124,16.263713,4.280116,0.833333,0.166667,NaN
EXC_L23,203,300.0,0.362077,0.247632,0.103153,2.477137,1.277312,1.389473,1.525084,13.299941,20.434890,3.897376,0.538961,0.285714,0.175325
EXC_L4,301,328.0,0.257630,0.173085,0.107508,2.831479,2.749493,3.520433,3.572248,26.940433,30.798646,55.895152,0.614973,0.299465,0.085561
EXC_L5,482,493.0,0.269807,0.116575,0.101203,3.560000,4.404760,5.682248,5.712125,34.169678,38.270261,4.596126,0.709877,0.231481,0.058642
EXC_L6,191,153.0,0.453816,0.125892,0.101152,3.386628,2.618949,3.522565,3.089944,34.154480,33.820845,5.051780,0.759494,0.177215,0.063291
PV_L23,75,58.0,0.134570,0.109628,0.099200,2.022556,5.400860,6.565192,7.934139,21.898555,25.463738,8.248520,0.666667,0.166667,0.166667
PV_L4,98,81.0,0.128351,0.090604,0.080612,2.474860,7.065440,9.273386,10.468564,29.372559,32.405075,8.761478,0.622222,0.222222,0.155556
PV_L5,99,101.0,0.142074,0.072370,0.098182,3.135000,9.377453,14.016601,11.958363,33.405722,38.479865,10.525396,0.790123,0.148148,0.061728
PV_L6,56,34.0,0.184120,0.074813,0.098214,3.311111,9.196765,14.364509,12.291614,46.731462,46.020400,7.618471,0.807692,0.153846,0.038462


In [8]:
print("SEM of functional responses")
resource_tables["sem"][selected_metrics]

SEM of functional responses


,n_cells_BO,n_cells_FC,g_osi_dg,g_dsi_dg,pref_sf_sg,pref_tf_dg,firing_rate_rf,firing_rate_dg,firing_rate_sp,width_rf,height_rf,fano_dg,frac_highpass,frac_bandpass,frac_lowpass
cell_type,,,,,,,,,,,,,,,
ALL_L1,50,91.0,0.043166,0.042689,0.007945,0.203024,0.173484,0.189598,1.109082,26.063112,14.634315,0.390963,0.152145,0.152145,NaN
EXC_L23,203,300.0,0.016712,0.012681,0.007460,0.118060,0.261655,0.097627,0.125160,4.984498,1.381575,0.242194,0.040169,0.036403,0.030641
EXC_L4,301,328.0,0.010779,0.009097,0.006329,0.130665,0.170568,0.166594,0.159461,1.960491,1.093955,51.607371,0.035584,0.033494,0.020455
EXC_L5,482,493.0,0.010439,0.005728,0.004796,0.134064,0.182511,0.149581,0.163045,0.971862,1.195018,0.194254,0.025212,0.023432,0.013053
EXC_L6,191,153.0,0.018166,0.009326,0.007977,0.217458,0.193778,0.159979,0.182125,2.226798,3.266223,0.278419,0.048085,0.042962,0.027394
PV_L23,75,58.0,0.009851,0.009462,0.011680,0.197808,0.501571,0.434693,0.487128,1.308490,1.674194,0.578864,0.078567,0.062113,0.062113
PV_L4,98,81.0,0.007612,0.007483,0.009258,0.188875,1.019920,0.483573,0.634559,1.529605,1.662128,0.663719,0.072274,0.061975,0.054028
PV_L5,99,101.0,0.009095,0.005450,0.009649,0.268878,0.899782,0.844381,0.782222,1.764641,1.997253,0.884059,0.045247,0.039472,0.026740
PV_L6,56,34.0,0.018088,0.008098,0.012906,0.436375,1.236104,1.261296,1.095721,3.397199,4.160537,0.670723,0.077292,0.070759,0.037715
